In [2]:
import pandas as pd 
import numpy as np
import lightgbm as lgb 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter('ignore')

#coding:utf8
import time
time0=time.time()
print('start_time时间戳:',time0)
from importlib import reload
from odps import ODPS
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder                 #分类变量数字化编码
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np
import json
from sklearn.ensemble import RandomForestClassifier            #随机森林
from sklearn.ensemble import GradientBoostingClassifier        #梯度上升
from sklearn.model_selection import train_test_split          #数据拆分
from sklearn.linear_model import LogisticRegression as LR      #逻辑回归
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_curve,auc                      #模型效果评估
from odps.df import DataFrame                                  #odps表数据读取
import information_value2 as iv                                 #导入iv/woe计算包
reload(iv)
from scipy import stats                                        #描述统计包
from sklearn.feature_selection import SelectFromModel          #模型特征选择
from odps.utils import init_progress_ui
import warnings
warnings.simplefilter('ignore')
time1=time.time()
# # 读取数据
# # def preProcess():
# path = 'data/'
# df_train = pd.read_csv(path + 'train.csv')
# df_test = pd.read_csv(path + 'test.csv')
# df_train.drop(['Id'], axis = 1, inplace = True)
# df_test.drop(['Id'], axis = 1, inplace = True)
# df_test['Label'] = -1
# data = pd.concat([df_train, df_test])
# data = data.fillna(-1)
# data.to_csv('data/data.csv', index = False)
# #     return data


def load_data(table_name):
    """加载数据集,数据预处理"""

    odps = ODPS("LTAI4GAgxSUBsvhqBpkwP4eR","Wjbt6SvdgH3vpJ925lp74DUQMkwWkL","wead")    #odps对象创建（连接设置）
    project = odps.get_project('wead')
#     odps_table = 'tmp_wax_1001_model_exp_act_kmyx_rmj_i' # 表名
    odps_table = table_name
    t = odps.get_table(odps_table)     # 获取某个表
    readers = t.open_reader()   # 读取表reader = t.open_reader(partition='pt=test')
    count = readers.count       # 读取表记录数
    print (u'%s 表记录数: %s'  %(odps_table,count))



    dd=pd.DataFrame()            #生成空表
    dd['name']=t.schema.names    #往表中加变量名称
    dd['type']=t.schema.types    #往表中加变量类型
    cc=dd[(dd['name']!='user_id') & (dd['name']!='p_date') & (dd['name']!='model_class') & (dd['name']!='user_veid') & (dd['name']!='is_act')& (dd['name']!='randnum')& (dd['name']!='rn1')&(dd['name']!='rn')]      #只取自变量名称和类型
    t_var_name = t.schema.names         #获取所有变量名称列表
    z_var_name=list(cc['name'])         #获取自变量名称列表 
    y_var_name='is_act'            #获取因变量名称
    u_var_name=t_var_name[0]                            #获取用户id
    s_var_name=list(cc['name'][cc['type']=='string'])   #获取分类变量名称
    v_var_name=list(cc['name'][cc['type']!='string'])   #获取数值变量名称


    time2=time.time()
    print('\n')
    print('数据导出前时间戳:',time2,'累计耗时：',time2-time0,'区间耗时：',time2-time1)

    print('\n')
    print ( u'用DataFrame结构存储数据，以便数据操作和建模(这个时间较久1)')
    ui = init_progress_ui(mock=True) 
    df1 = DataFrame(t).to_pandas(ui=ui)  

    time3=time.time()
    print('数据导出后时间戳:',time3,'累计耗时：',time3-time0,'区间耗时：',time3-time2)
    print('\n')
    return df1,s_var_name,v_var_name,z_var_name,y_var_name


df1,s_var_name,v_var_name,z_var_name,y_var_name = load_data('tmp_wax_1001_model_flag_xn_yx_jg_c')

print(z_var_name)

start_time时间戳: 1616221464.080758
tmp_wax_1001_model_flag_xn_yx_jg_c 表记录数: 300000


数据导出前时间戳: 1616221466.0305872 累计耗时： 1.9498291015625 区间耗时： 0.9983372688293457


用DataFrame结构存储数据，以便数据操作和建模(这个时间较久1)
数据导出后时间戳: 1616222023.2481108 累计耗时： 559.1673526763916 区间耗时： 557.2175235748291


['os', 'agerange', 'gender', 'car', 'house', 'child', 'position', 'love', 'educate', 'interest_101201', 'interest_101103', 'interest_101102', 'interest_101101', 'interest_101004', 'interest_101003', 'interest_101002', 'interest_101001', 'interest_100102', 'interest_100101', 'interest_101902', 'interest_101804', 'interest_101805', 'interest_101903', 'interest_101801', 'interest_101803', 'interest_101901', 'interest_101704', 'interest_101802', 'interest_101701', 'interest_101702', 'interest_101703', 'interest_101601', 'interest_101603', 'interest_101604', 'interest_101503', 'interest_101504', 'interest_101602', 'interest_10160x', 'interest_101501', 'interest_101502', 'interest_10150x', 'interest_101404', 'interest_10

In [ ]:
def preprocess():
    
     #缺失值填充
    df1[s_var_name]=df1[s_var_name].fillna(value='未知')                 #（分类）自变量缺失值填充
    df1[v_var_name]=df1[v_var_name].fillna(df1[v_var_name].mean())       #数值自变量以均值填充

    time4=time.time()
#     print('\n')
#     print('数据缺失值填充时间戳:',time4,'累计耗时：',time4-time0,'区间耗时：',time4-time3)

    #各变量缺失值替换值
    default_var = []   
    for var in z_var_name:
        if var in s_var_name:
            default_var.append('未知')
        else:
            default_var.append(df1[var].mean())

    s_default_var=pd.Series(default_var,index=z_var_name)
    
    
    # 对分类自变量进行编码
    z_var_dict={}
    for var in z_var_name:     
        num = LabelEncoder()
        df1[var] = num.fit_transform(df1[var].astype('str'))
        dict0={}
        for label in num.classes_:
            dict0[label]=num.transform([label])[0]
        z_var_dict[var]=dict0

    time5=time.time()
    print('数据缺失值保存时间戳:',time5,'累计耗时：',time5-time0,'区间耗时：',time5-time4)


    

    # 变量分段
    # 对数值变量进行分箱
    df2=df1.copy()
    iva=iv.WOE()
    x_cate_max_rate_dict,x_cate_max_rate_array=iva.x_cate_max_rate(df2[z_var_name], df2[y_var_name], event=1)
    z_var_name0=list(np.array(z_var_name)[x_cate_max_rate_array>0.9])
    z_var_name1=list(set(z_var_name)-set(z_var_name0))  
    v_var_name1=list(set(z_var_name1).intersection(set(v_var_name)))  
    s_var_name1=list(set(z_var_name1).intersection(set(s_var_name)))  


    df2[v_var_name1],v_var_dicrete=iva.feature_discretion(df2[v_var_name1])            #数值变量分箱并替换
    v_dict=dict(zip(v_var_name1,v_var_dicrete))                                       #数值变量分段标准字典

    time6=time.time()
    print('\n')
    print('对数值变量分箱时间戳:',time6,'累计耗时：',time6-time0,'区间耗时：',time6-time5)
    # 对分类变量进行分箱
    #计算分类变量iv值及woe，对变量进行WOE转换后再进行分箱处理
    w_dict,i_dict,w_list,i_list=iva.woe(df2[s_var_name1], df2[y_var_name], event=1)    #(这个时间较久)

    print('\n')
    print ( u'计算分类变量iv值及woe，对变量进行WOE转换后再进行分箱处理(这个时间较久2)')
    df2[s_var_name1]=iva.woe_replace(df2[s_var_name1],w_list)                    #对变量进行WOE转换     #(这个时间较久)
    df2[s_var_name1],s_var_dicrete=iva.feature_discretion(df2[s_var_name1])      #分类变量分箱并替换
    s_dict=dict(zip(s_var_name1,s_var_dicrete))               #分类变量分段标准字典       
    s_var_dicrete_dict=iva.exchage_dict(s_dict,w_dict)       #分类变量分段映射关系字典
    # 合并分类变量与数值变量的分箱数据
    z_var_dicrete_dict=s_var_dicrete_dict.copy()
    z_var_dicrete_dict.update(v_dict)                        

    time7=time.time()
    #print('对分类变量分箱时间戳:',time7,'累计耗时：',time7-time0,'区间耗时：',time7-time6)
    # 对分箱后的数据进行数据类型转换
    for var in z_var_name1:
        df2[var] = df2[var].astype(str)
    time8=time.time()
    #print('对分箱后的数据进行str转换时间戳:',time8,'累计耗时：',time8-time0,'区间耗时：',time8-time7)
    #训练集/测试集划分
#     point1= stats.scoreatpercentile(df2['p_date'], 70)  
#     point2= stats.scoreatpercentile(df2['p_date'], 75) 
#     if  point1==point2:
    x_train,x_test, y_train, y_test = train_test_split(df2[z_var_name1], df2[y_var_name], test_size = 0.3, random_state=2020)      #默认按1:3划分训练和测试样本
    print (u'按比例随机划分训练集/测试集')
#     else:
#         df_train=df2[df2['p_date']<=point1]
#         df_test=df2[df2['p_date']>point1]
#         x_train=df_train[z_var_name1]
#         x_test=df_test[z_var_name1]
#         y_train=df_train[y_var_name]
#         y_test=df_test[y_var_name]
#         print (u'按时间先后划分训练集/测试集')
#     print('\n')
#     print('-' * 60)
#     print ('point1:' , point1)
#     print('-' * 60)

    time9=time.time()
    print('\n')
    print('对数据进行划分数据集时间戳:',time9,'累计耗时：',time9-time0,'区间耗时：',time9-time8)
    #对分箱后的所有自变量计算iv值及woe，并进行WOE转换，再根据iv值进行变量预筛选
    w_dict1,i_dict1,w_list1,i_list1=iva.woe(df2[z_var_name1], df2[y_var_name], event=1)     #(这个时间较久)

    time10=time.time()

    z_var_woe_dict=iva.exchage_func(z_var_dicrete_dict,w_dict1)      #变量各段类与对应WOE的映射关系字典

    # 对变量进行WOE转换
    x_train[z_var_name1]=iva.woe_replace(x_train[z_var_name1],w_list1)
    x_test[z_var_name1]=iva.woe_replace(x_test[z_var_name1],w_list1)

    time11=time.time()


    # 根据iv值筛选入模训练变量
    n_features = 15
    iv_df=pd.Series(i_list1,index=x_train.columns)
    s_iv_df=iv_df.nlargest(n_features)
    s_iv_name=list(s_iv_df.index)
    print('-' * 60)
    print(u'前%d个s_iv_df'%n_features)
    print(s_iv_df)
    print('\n')


    
    
    
    #     类别特征one-hot编码
    for col in s_var_name:
        onehot_feats = pd.get_dummies(df2[col], prefix = col)
        df2.drop([col], axis = 1, inplace = True)
        df2 = pd.concat([df2, onehot_feats], axis = 1)

    x_train,x_val,y_train,y_val = train_test_split(df2,df2[y_var_name],test_size=0.3,random_state=2021)
    print(x_train.head())
    return x_train,x_val,y_train,y_val,s_iv_name
    
    
x_train,x_val,y_train,y_val,s_iv_name = preprocess()
data = x_train





In [ ]:
def gbdt_lr_predict(data): 

    n_estimators = 50
    num_leaves = 64
    # 开始训练gbdt，使用100课树，每课树64个叶节点
    model = lgb.LGBMRegressor(objective='binary',
                            subsample= 0.8,
                            min_child_weight= 0.5,
                            colsample_bytree= 0.7,
                            num_leaves=num_leaves,
                            learning_rate=0.05,
                            n_estimators=n_estimators,
                            random_state = 2020)
    model.fit(x_train, y_train,
            eval_set = [(x_train, y_train), (x_val, y_val)],
            eval_names = ['train', 'val'],
            eval_metric = 'binary_logloss',
            verbose=0)
    
    # 得到每一条训练数据落在了每棵树的哪个叶子结点上
    # pred_leaf = True 表示返回每棵树的叶节点序号
    gbdt_feats_train = model.predict(train, pred_leaf = True)
    
    # 打印结果的 shape：
    print(gbdt_feats_train.shape)
    # 打印前5个数据：
    print(gbdt_feats_train[:5])
    
    # 同样要获取测试集的叶节点索引
    gbdt_feats_test = model.predict(test, pred_leaf = True)
    
    # 将 32 课树的叶节点序号构造成 DataFrame，方便后续进行 one-hot
    gbdt_feats_name = ['gbdt_leaf_' + str(i) for i in range(n_estimators)]
    df_train_gbdt_feats = pd.DataFrame(gbdt_feats_train, columns = gbdt_feats_name) 
    df_test_gbdt_feats = pd.DataFrame(gbdt_feats_test, columns = gbdt_feats_name)
    train_len = df_train_gbdt_feats.shape[0]
    data = pd.concat([df_train_gbdt_feats, df_test_gbdt_feats])

    # 对每棵树的叶节点序号进行 one-hot
    for col in gbdt_feats_name:
        onehot_feats = pd.get_dummies(data[col], prefix = col)
        data.drop([col], axis = 1, inplace = True)
        data = pd.concat([data, onehot_feats], axis = 1)

    train = data[: train_len]
    test = data[train_len:]
    
    # 划分 LR 训练集、验证集
    x_train, x_val, y_train, y_val = train_test_split(train, target, test_size = 0.3, random_state = 2018)
    
    # 开始训练lr
    lr = LogisticRegression()
    lr.fit(x_train, y_train)
    tr_logloss = log_loss(y_train, lr.predict_proba(x_train)[:, 1])
    tr_auc = roc_auc_score(y_train,lr.predict_proba(x_train)[:,1])
#     print('tr-logloss: ', tr_logloss)
    print('train_auc:',tr_auc)
    val_logloss = log_loss(y_val, lr.predict_proba(x_val)[:, 1])
    test_auc  = roc_auc_score(y_val,lr.predict_proba(x_val)[:,1])
#     print('val-logloss: ', val_logloss)
    print('test_auc:',test_auc)
    # 对测试集预测
    y_pred = lr.predict_proba(test)[:, 1]
#     test_val = pd.concat(df_test['Id'],y_ped)
    print(len(y_pred))
    
    


# if __name__ == '__main__':
#     data = preProcess()
#     continuous_feature = ['I'] * 13
#     continuous_feature = [col + str(i + 1) for i, col in enumerate(continuous_feature)] 
#     category_feature = ['C'] * 26
#     category_feature = [col + str(i + 1) for i, col in enumerate(category_feature)] 
#     gbdt_lr_predict(data, category_feature, continuous_feature)
#     print(continuous_feature,category_feature)

In [ ]:
gbdt_lr_predict(data)